In [1]:
%%capture
import os

if os.getenv("COLAB_RELEASE_TAG"):
    !git clone https://github.com/ivanbrillo/tesiRFX
    %cd tesiRFX
    !rsync -a --exclude='*.ipynb' --exclude='*.git' ./ ../
    %cd ..
    !rm -rf tesiRFX
    
import tensorflow as tf
from tensorflow.keras import layers
import customLib.autoencoder_helper as helper
from customLib.betaVAE import VAE
from customLib.AE import AE, get_sequentials_outer

In [2]:
train, test = helper.get_splitted_np(win_size=80, alpha=40)

In [3]:
autoencoder = AE(*get_sequentials_outer())
autoencoder.set_trainable(False)
autoencoder.build(input_shape=(None, 1800, 1))
autoencoder.load_weights("./weights/convAE.h5")

In [4]:
def get_seq_fullAE():
    encoder = tf.keras.Sequential([
        autoencoder.encoder,
        layers.Dense(70, activation='linear'),
        layers.Dense(50, activation='relu'),
        layers.Dense(30, activation='relu'),
        layers.Dense(10, activation='linear'),
    ])

    decoder = tf.keras.Sequential([
        layers.Dense(10, activation='linear'),
        layers.Dense(30, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(70, activation='relu'),
        layers.Dense(80, activation='linear'),
        autoencoder.decoder,
    ])

    return encoder, decoder


fullVAE = VAE(*get_seq_fullAE(), latent_dim=2)

In [6]:
helper.train_and_evaluate(fullVAE, train, test, epochs_n=5000, apply_filter=True, show_latent=True, patience=500, batch_size=400, monitor="val_mse")

[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0), (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0), (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0), (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0), (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0), (0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354, 1.0), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354, 1.0), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354, 1.0), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354, 1.0), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354, 1.0), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354, 1.0), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354, 1.0), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354, 1.0), (0.5490196078431373, 0.3372549019607843

AssertionError: in user code:

    File "C:\Users\brill\Documents\GitHub\tesiRFX\venv\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\brill\Documents\GitHub\tesiRFX\venv\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\brill\Documents\GitHub\tesiRFX\venv\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)

    AssertionError: Called a function referencing variables which have been deleted. This likely means that function-local variables were created and not referenced elsewhere in the program. This is generally a mistake; consider storing variables in an object attribute on first call.
